いろいろ確認

- データをグループ化
- データを可視化

In [1]:
import pandas as pd
import altair as alt
import titanite as ti

print(f"Pandas {pd.__version__}")
print(f"Altair {alt.__version__}")
print(f"Titanite {ti.__version__}")

Pandas 2.0.3
Altair 5.0.1
Titanite 0.1.1


In [2]:
fcfg = "../sandbox/config.toml"
fcsv = "../sandbox/tmp_preprocessed.csv"

cfg = ti.Config(fname=fcfg)
cfg.load()
category = cfg.categories()

data = pd.read_csv(fcsv, parse_dates=["timestamp"])
data = ti.categorical_data(data, category)
# data

cfg.questions

{'q1': '【Q1】What is your age ?',
 'q2': '【Q2】What gender do you identify as ?',
 'q3': '【Q3】Which geographical region are you currently working or attending school/university in ?',
 'q4': '【Q4】Which geographical region do you most strongly associate with ?',
 'q5': '【Q5】What is your job title ?',
 'q6': '【Q6】Which group do you belong to ? ',
 'q7': '【Q7】What is your research type ?',
 'q8': '【Q8】How long have you been in this field ?',
 'q9': '【Q9】Are you satisfied with your career to date ?',
 'q10': '【Q10】How many hours, on average, do you spend on housework, childcare, and caregiving per day ?',
 'q11': '【Q11】Did you already sign up for the diversity session in ICRC2023?',
 'q12': '【Q12】What do you think about the initiatives on DE&I of your group?',
 'q13': '【Q13】What is the percentage of female researcher in your group?',
 'q14': '【Q14】What do you think about the percentage above ?',
 'q15': '【Q15】Please let us know If your group has any good practice examples related to DE&I ?',

In [3]:
def group_data(data: pd.DataFrame, x: str, y: str):
    names = [x, y]
    c = "response"
    grouped = data.groupby(names)[c].sum().reset_index()
    return grouped

In [4]:
def histogram(data: pd.DataFrame, x: str, color: str):
    y = "response"
    h = alt.Chart(data).mark_bar().encode(
        alt.X(x),
        alt.Y(y),
        alt.Color(color),
    ).properties(
        width = 500,
    )
    return h    

In [5]:
def heatmap(data: pd.DataFrame, x: str, y: str, color: str):
    hm = alt.Chart(data).mark_rect().encode(
        alt.X(x),
        alt.Y(y),
        alt.Color(color),
    ).properties(
        width = 500,
    )
    return hm

In [91]:
x = "q2"
y = "q6"
z = "response"

gdata = group_data(data, x, y)
hi = histogram(gdata, x, y)
hm = heatmap(gdata, x, y, color="response")

In [93]:
hi & hm

alt.VConcatChart(...)

In [6]:
from vega_datasets import data as vdata

In [7]:
data["q4_regional"]

0       Europe
1         Asia
2         Asia
3      America
4      America
        ...   
255     Europe
256       Asia
257       Asia
258     Europe
259     Europe
Name: q4_regional, Length: 260, dtype: category
Categories (6, object): ['Europe' < 'Asia' < 'America' < 'Oceania' < 'Africa' < 'Prefer not to answer']

In [10]:
source = alt.topo_feature(vdata.world_110m.url, 'countries')
base = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='blue'
).properties(
    width=500,
    height=500
)
.project(
    "mercator"
).display()
#projections = ['equirectangular', 'mercator', 'orthographic', 'gnomonic']
#charts = [base.project(proj).properties(title=proj) for proj in projections]
#alt.concat(*charts, columns=2)

alt.Chart(...)

In [11]:
alt.Chart(data).mark_geoshape().encode(
    color='q4_regional:N'
)

alt.Chart(...)